### TAKE OUT 80 IMAGES FROM BLACK AND ORANGE CARS

In [6]:
import os
import shutil

def select_and_save_images(directory_path, output_image_path, output_npy_path, npy_directory_path, name, num_files=80):
    """
    Selects 'num_files' number of image files from the given directory in equal intervals.
    The selected image files are saved to one directory, and the corresponding '.npy' files
    are matched with the image files and saved to another directory.

    Parameters:
    - directory_path: Path to the directory containing the image files.
    - output_image_path: Path to the directory where the selected image files will be stored.
    - output_npy_path: Path to the directory where the corresponding '.npy' files will be stored.
    - npy_directory_path: Path to the directory containing the '.npy' files.
    - num_files: Number of image files to select.

    Returns:
    - List of file names (without extensions) of the selected image files.
    """
    if not os.path.exists(output_image_path):
        os.makedirs(output_image_path)

    if not os.path.exists(output_npy_path):
        os.makedirs(output_npy_path)

    file_list = os.listdir(directory_path)
    image_extensions = ['.jpg', '.jpeg', '.png']  # Add other image extensions if needed

    image_files = [file for file in file_list if any(file.lower().endswith(ext) for ext in image_extensions)]
    image_files.sort()  # Sort image files for consistency

    total_files = len(image_files)

    if num_files >= total_files:
        raise ValueError("Number of files to select is greater than or equal to the total number of image files.")

    interval = total_files // num_files
    selected_image_files = []

    for i in range(0, total_files, interval):
        file_name, file_extension = os.path.splitext(image_files[i])
        selected_image_files.append(file_name)

        # Save selected image files
        image_file_path = os.path.join(directory_path, image_files[i])
        output_image_file_path = os.path.join(output_image_path, name + image_files[i])
        shutil.copy(image_file_path, output_image_file_path)

        # Matching with corresponding '.npy' files and save
        npy_file_path = os.path.join(npy_directory_path, name + file_name + ".npy")
        # Assuming there is a corresponding '.npy' file for each image
        # Replace this with the actual path or logic to get '.npy' files
        shutil.copy(npy_file_path, os.path.join(output_npy_path, name + file_name + ".npy"))

    return selected_image_files



# Example usage:
# Example usage:
black_path = "./data/carseg_data/unused/images/black_5_doors/no_segmentation"
orange_path = "./data/carseg_data/unused/images/orange_3_doors/no_segmentation"
output_path = "./data/carseg_data/clean_arrays"
output_im_path = "./data/carseg_data/clean_arrays_images"
npy_directory_path = "./data/carseg_data/unused/arrays"

selected_image_files = select_and_save_images(black_path, output_im_path, output_path, npy_directory_path, "black_5_doors_", num_files=80)
selected_image_files = select_and_save_images(orange_path, output_im_path, output_path, npy_directory_path, "orange_3_doors_",num_files=80)

print("Selected image files:", selected_image_files)


Selected image files: ['0001', '0026', '0051', '0076', '0101', '0126', '0151', '0176', '0201', '0226', '0251', '0276', '0301', '0326', '0351', '0376', '0401', '0426', '0451', '0476', '0501', '0526', '0551', '0576', '0601', '0626', '0651', '0676', '0701', '0726', '0751', '0776', '0801', '0826', '0851', '0876', '0901', '0926', '0951', '0976', '1001', '1026', '1051', '1076', '1101', '1126', '1151', '1176', '1201', '1226', '1251', '1276', '1301', '1326', '1351', '1376', '1401', '1426', '1451', '1476', '1501', '1526', '1551', '1576', '1601', '1626', '1651', '1676', '1701', '1726', '1751', '1776', '1801', '1826', '1851', '1876', '1901', '1926', '1951', '1976', '2001']
